<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/476_MO_nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Mission Orchestrator Nodes — Architectural Explanation

## What These Nodes Represent

These nodes define the **end-to-end execution lifecycle** of a business mission.

Each node is a **single-responsibility decision point** in the system — responsible for one clear phase of mission execution — and together they form a deterministic, auditable workflow from *goal definition* to *final executive report*.

This is not a conversational agent.
This is a **governed execution pipeline**.

---

## Why a Node-Based Architecture Matters

Most AI systems blur phases together:

* Planning happens implicitly
* Execution decisions are opaque
* Progress is inferred after the fact
* Reporting is bolted on at the end

Your node design does the opposite.

Each phase:

* Has a name
* Has inputs
* Produces outputs
* Writes explicitly to shared state

That structure is what allows the system to be:

* Explainable
* Testable
* Auditable
* Extendable

Executives don’t have to “trust the AI” — they can inspect the workflow.

---

## How the Nodes Work Together (Big Picture)

Conceptually, the system follows this lifecycle:

**Define → Plan → Load → Order → Execute → Track → Evaluate → Govern → Report**

Each node corresponds to one of those verbs.

---

## Node-by-Node System Role

### 1. Goal Node — Declaring Intent

The goal node establishes **why the mission exists**.

Rather than generating an open-ended objective, it:

* Anchors execution to a specific mission ID
* Defines focus areas explicitly
* Creates a timestamped record of intent

This ensures the system never executes “just because it can.”

From a leadership standpoint, this answers:

> “What was this system trying to achieve?”

---

### 2. Planning Node — Making Execution Explicit

The planning node converts intent into a **clear execution blueprint**.

Each step:

* Has a name
* Declares dependencies
* Defines expected outputs

This is critical: the system knows *what will happen next* before anything runs.

There is no improvisation here — only structured execution.
That predictability is a core trust signal.

---

### 3. Data Loading Node — Establishing the Rules of the Mission

This node pulls in:

* Mission definition
* Tasks
* KPIs
* Available agents
* Capability constraints
* Optional execution context

At this point, the system has everything it needs to operate — and **nothing more**.

Importantly, all of this comes from configuration, not code.
That’s how strategy remains editable without redeployment.

---

### 4. Task Ordering Node — Enforcing Dependencies

Here the system:

* Resolves task dependencies
* Detects circular logic
* Builds an initial execution queue

This ensures:

* Tasks run in the correct order
* No hidden assumptions exist
* Execution failures are structural, not mysterious

This mirrors how real operational plans are validated before execution.

---

### 5. Task Execution Node — Controlled Action

This node is where work actually happens.

Key characteristics:

* Tasks only execute when dependencies are satisfied
* Agent selection is capability-based
* Execution results are recorded in full
* The queue is updated deterministically

There is no “agent autonomy” here.
There is **mission discipline**.

Every action produces a traceable execution record.

---

### 6. Progress Tracking Node — Operational Visibility

Rather than guessing progress, this node calculates it explicitly:

* Percentage complete
* Elapsed time
* Estimated remaining time

This turns execution into something leaders can monitor in real time.

Instead of:

> “The system is running”

You get:

> “We are 60% complete, on schedule, with 12 minutes remaining.”

That’s operational maturity.

---

### 7. KPI Calculation Node — Measuring Outcomes, Not Activity

This node ensures the mission isn’t just *done* — it’s **effective**.

It:

* Computes KPI metrics based on executed tasks
* Assesses performance against thresholds
* Flags risk before failure

This is where the orchestrator stops being a workflow engine and becomes a **business system**.

Execution without evaluation is automation.
Execution with KPIs is management.

---

### 8. Approval Workflow Node — Human Governance

Human-in-the-loop isn’t an afterthought here — it’s a first-class phase.

This node:

* Identifies tasks requiring approval
* Tracks pending decisions
* Supports auto-approval for testing
* Records approval history

This creates:

* Accountability
* Safety
* Auditability

The system knows *when* humans are required — and *why*.

---

### 9. Report Generation Node — Executive Closure

The final node produces a **complete mission narrative**.

It:

* Generates a structured report
* Saves it with traceable identifiers
* Determines mission status
* Records completion reasons

The mission doesn’t just “end” — it **concludes with evidence**.

This is what turns AI execution into something leaders can review, share, and defend.

---

## Why This Node Design Is Enterprise-Grade

This architecture demonstrates:

* **Separation of concerns** — each node has one job
* **Determinism** — no hidden branching logic
* **Traceability** — every decision leaves a record
* **Governance** — approvals and thresholds are explicit
* **Scalability** — new nodes can be added without rewriting the system

Most agent demos stop at “it worked.”
This system answers:

* Why it worked
* Whether it should have
* What to change next time

---

## Bottom Line

These nodes form the **operating spine** of your Mission Orchestrator.

They transform:

* Business intent → structured plan
* Plan → controlled execution
* Execution → measurable outcomes
* Outcomes → executive insight

This is not experimental AI.
This is **AI-driven operations**.



In [ ]:
"""Mission Orchestrator Agent Nodes

Nodes for the Mission Orchestrator workflow.
Following MVP-first approach: rule-based foundation, no LLM initially.
"""

from typing import Dict, Any
from datetime import datetime
from config import MissionOrchestratorState
from agents.mission_orchestrator.utilities.data_loading import (
    load_mission_data,
    load_mission_tasks,
    load_mission_kpis,
    load_specialized_agents,
    load_capabilities_matrix,
    build_agent_lookup,
    build_capabilities_lookup,
    load_execution_context
)
from toolshed.tasks import resolve_task_dependencies, get_ready_tasks
from agents.mission_orchestrator.utilities.agent_execution import execute_next_task
from toolshed.progress import (
    calculate_progress,
    calculate_elapsed_time,
    estimate_remaining_time
)
from toolshed.kpi import (
    calculate_kpi_metrics,
    assess_kpi_status
)
from toolshed.hitl import (
    get_pending_approvals,
    auto_approve_for_testing,
    is_task_approved
)
from toolshed.reporting import generate_mission_report, save_report


def goal_node(state: MissionOrchestratorState) -> Dict[str, Any]:
    """
    Goal Node: Define the goal for mission execution.

    This is a simple rule-based goal definition that sets the framework
    for executing a business mission.
    """
    mission_id = state.get("mission_id")
    errors = state.get("errors", [])

    if not mission_id:
        return {
            "errors": errors + ["goal_node: mission_id is required"]
        }

    goal = {
        "objective": f"Execute mission {mission_id} to achieve business outcomes",
        "mission_id": mission_id,
        "focus_areas": [
            "task_execution",
            "agent_coordination",
            "progress_tracking",
            "kpi_optimization",
            "human_approval_workflows"
        ],
        "created_at": datetime.now().isoformat()
    }

    return {
        "goal": goal,
        "errors": errors
    }


def planning_node(state: MissionOrchestratorState) -> Dict[str, Any]:
    """
    Planning Node: Create execution plan based on goal.

    This creates a step-by-step plan for mission execution.
    Rule-based, no LLM needed for MVP.
    """
    goal = state.get("goal")
    errors = state.get("errors", [])

    if not goal:
        return {
            "errors": errors + ["planning_node: goal is required"]
        }

    plan = [
        {
            "step": 1,
            "name": "data_loading",
            "description": "Load mission data, tasks, agents, and KPIs",
            "dependencies": [],
            "outputs": ["mission", "mission_tasks", "specialized_agents", "capabilities_matrix", "mission_kpis"]
        },
        {
            "step": 2,
            "name": "task_ordering",
            "description": "Order tasks by dependencies and resolve execution order",
            "dependencies": ["data_loading"],
            "outputs": ["task_queue", "tasks_total"]
        },
        {
            "step": 3,
            "name": "task_execution",
            "description": "Execute tasks sequentially, assigning to appropriate agents",
            "dependencies": ["task_ordering"],
            "outputs": ["executed_tasks", "task_results"]
        },
        {
            "step": 4,
            "name": "progress_tracking",
            "description": "Track mission progress and calculate metrics",
            "dependencies": ["task_execution"],
            "outputs": ["progress_percentage", "tasks_completed", "elapsed_time_minutes", "estimated_remaining_minutes"]
        },
        {
            "step": 5,
            "name": "kpi_calculation",
            "description": "Calculate KPI metrics and assess status",
            "dependencies": ["progress_tracking"],
            "outputs": ["kpi_metrics", "kpi_status"]
        },
        {
            "step": 6,
            "name": "approval_workflow",
            "description": "Handle human-in-the-loop approvals for tasks requiring approval",
            "dependencies": ["task_execution"],
            "outputs": ["pending_approvals", "approval_history"]
        },
        {
            "step": 7,
            "name": "report_generation",
            "description": "Generate final mission execution report",
            "dependencies": ["kpi_calculation", "approval_workflow"],
            "outputs": ["mission_report", "report_file_path"]
        }
    ]

    return {
        "plan": plan,
        "errors": errors
    }


def data_loading_node(state: MissionOrchestratorState) -> Dict[str, Any]:
    """
    Data Loading Node: Orchestrate loading mission data, tasks, agents, and KPIs.
    """
    errors = state.get("errors", [])
    mission_id = state.get("mission_id")
    data_dir = state.get("data_dir", "agents/data")

    if not mission_id:
        return {
            "errors": errors + ["data_loading_node: mission_id is required"]
        }

    try:
        # Load mission data
        mission = load_mission_data(mission_id, data_dir)

        # Load mission tasks
        mission_tasks = load_mission_tasks(mission_id, data_dir)

        # Load mission KPIs
        mission_kpis = load_mission_kpis(mission_id, data_dir)

        # Load agents and capabilities
        specialized_agents = load_specialized_agents(data_dir)
        capabilities_matrix = load_capabilities_matrix(data_dir)

        # Build lookup dictionaries for fast access
        agents_lookup = build_agent_lookup(specialized_agents)
        capabilities_lookup = build_capabilities_lookup(capabilities_matrix)

        # Load execution context (optional)
        execution_context = load_execution_context(mission_id, data_dir)

        return {
            "mission": mission,
            "mission_tasks": mission_tasks,
            "mission_kpis": mission_kpis,
            "specialized_agents": specialized_agents,
            "capabilities_matrix": capabilities_matrix,
            "agents_lookup": agents_lookup,
            "capabilities_lookup": capabilities_lookup,
            "execution_context": execution_context,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"data_loading_node: {str(e)}"]
        }


def task_ordering_node(state: MissionOrchestratorState) -> Dict[str, Any]:
    """
    Task Ordering Node: Order tasks by dependencies and prepare execution queue.
    """
    errors = state.get("errors", [])
    mission_tasks = state.get("mission_tasks", [])

    if not mission_tasks:
        return {
            "errors": errors + ["task_ordering_node: mission_tasks is required"]
        }

    try:
        # Resolve task dependencies and order tasks
        ordered_tasks = resolve_task_dependencies(mission_tasks)

        # Get ready tasks (tasks with no dependencies or all dependencies satisfied)
        # Initially, no tasks are completed, so ready tasks are those with no dependencies
        completed_task_ids = set()
        ready_tasks = get_ready_tasks(ordered_tasks, completed_task_ids)

        return {
            "task_queue": ready_tasks,
            "tasks_total": len(mission_tasks),
            "tasks_completed": 0,
            "errors": errors
        }
    except ValueError as e:
        # Circular dependency detected
        return {
            "errors": errors + [f"task_ordering_node: {str(e)}"]
        }
    except Exception as e:
        return {
            "errors": errors + [f"task_ordering_node: {str(e)}"]
        }


def task_execution_node(state: MissionOrchestratorState) -> Dict[str, Any]:
    """
    Task Execution Node: Execute tasks sequentially, assigning to appropriate agents.

    MVP: Executes all tasks in order. In production, this could support parallel execution.
    """
    errors = state.get("errors", [])
    task_queue = state.get("task_queue", [])
    executed_tasks = state.get("executed_tasks", [])
    capabilities_lookup = state.get("capabilities_lookup", {})
    agents_lookup = state.get("agents_lookup", {})
    execution_context = state.get("execution_context")
    data_dir = state.get("data_dir", "agents/data")
    mission_start_time = state.get("mission_start_time")

    # Set mission start time if not set
    if not mission_start_time:
        mission_start_time = datetime.now().isoformat()

    # Track completed task IDs for dependency resolution
    completed_task_ids = {task.get("task_id") for task in executed_tasks if task.get("status") == "completed"}

    # Execute tasks until queue is empty
    task_results = state.get("task_results", {})

    while task_queue:
        # Get ready tasks (dependencies satisfied)
        ready_tasks = get_ready_tasks(task_queue, completed_task_ids)

        if not ready_tasks:
            # No tasks ready (waiting on dependencies or all done)
            break

        # Execute next ready task
        execution_record = execute_next_task(
            ready_tasks,
            executed_tasks,
            capabilities_lookup,
            agents_lookup,
            execution_context,
            data_dir
        )

        if execution_record:
            executed_tasks.append(execution_record)
            task_id = execution_record.get("task_id")
            task_results[task_id] = execution_record.get("result", {})

            # Remove executed task from queue
            task_queue = [t for t in task_queue if t.get("task_id") != task_id]

            # Update completed task IDs if task succeeded
            if execution_record.get("status") == "completed":
                completed_task_ids.add(task_id)
        else:
            # No task could be executed (shouldn't happen, but handle gracefully)
            break

    # Calculate tasks completed
    tasks_completed = len([t for t in executed_tasks if t.get("status") == "completed"])

    return {
        "executed_tasks": executed_tasks,
        "task_queue": task_queue,
        "task_results": task_results,
        "tasks_completed": tasks_completed,
        "mission_start_time": mission_start_time,
        "errors": errors
    }


def progress_tracking_node(state: MissionOrchestratorState) -> Dict[str, Any]:
    """
    Progress Tracking Node: Track mission progress and calculate metrics.
    """
    errors = state.get("errors", [])
    tasks_completed = state.get("tasks_completed", 0)
    tasks_total = state.get("tasks_total", 0)
    mission_start_time = state.get("mission_start_time")
    executed_tasks = state.get("executed_tasks", [])
    task_queue = state.get("task_queue", [])

    # Calculate progress percentage
    progress = calculate_progress(tasks_completed, tasks_total)

    # Calculate elapsed time
    elapsed = 0.0
    if mission_start_time:
        elapsed = calculate_elapsed_time(mission_start_time)

    # Estimate remaining time
    remaining = estimate_remaining_time(executed_tasks, task_queue)

    return {
        "progress_percentage": progress,
        "elapsed_time_minutes": elapsed,
        "estimated_remaining_minutes": remaining,
        "errors": errors
    }


def kpi_calculation_node(state: MissionOrchestratorState) -> Dict[str, Any]:
    """
    KPI Calculation Node: Calculate KPI metrics and assess status.
    """
    errors = state.get("errors", [])
    executed_tasks = state.get("executed_tasks", [])
    mission_kpis = state.get("mission_kpis", {})

    if not mission_kpis:
        return {
            "errors": errors + ["kpi_calculation_node: mission_kpis is required"]
        }

    try:
        # Calculate KPI metrics
        kpi_metrics = calculate_kpi_metrics(executed_tasks, mission_kpis)

        # Assess KPI status
        kpi_status = assess_kpi_status(
            kpi_metrics,
            mission_kpis,
            warning_threshold=0.8,
            critical_threshold=0.5
        )

        return {
            "kpi_metrics": kpi_metrics,
            "kpi_status": kpi_status,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"kpi_calculation_node: {str(e)}"]
        }


def approval_workflow_node(state: MissionOrchestratorState) -> Dict[str, Any]:
    """
    Approval Workflow Node: Handle human-in-the-loop approvals for tasks requiring approval.
    """
    errors = state.get("errors", [])
    executed_tasks = state.get("executed_tasks", [])
    approval_history = state.get("approval_history", [])
    auto_approve = state.get("auto_approve_for_testing", True)

    # Get pending approvals
    pending_approvals = get_pending_approvals(approval_history, executed_tasks)

    # Auto-approve for testing if configured
    if auto_approve and pending_approvals:
        auto_approvals = auto_approve_for_testing(pending_approvals, auto_approve=True)
        approval_history = approval_history + auto_approvals
        pending_approvals = []

    return {
        "pending_approvals": pending_approvals,
        "approval_history": approval_history,
        "errors": errors
    }


def report_generation_node(state: MissionOrchestratorState) -> Dict[str, Any]:
    """
    Report Generation Node: Generate final mission execution report.
    """
    errors = state.get("errors", [])
    mission = state.get("mission", {})
    mission_id = mission.get("mission_id") or state.get("mission_id", "unknown")
    reports_dir = state.get("reports_dir", "output/mission_reports")

    try:
        # Generate report using toolshed
        report_content = generate_mission_report(state)

        # Save report
        report_file_path = save_report(
            report_content,
            mission_id,
            reports_dir=reports_dir,
            prefix="mission_report"
        )

        # Determine mission status
        tasks_completed = state.get("tasks_completed", 0)
        tasks_total = state.get("tasks_total", 0)
        pending_approvals = state.get("pending_approvals", [])

        if tasks_completed == tasks_total and not pending_approvals:
            mission_status = "completed"
            completion_reason = "All tasks completed successfully"
        elif pending_approvals:
            mission_status = "awaiting_approval"
            completion_reason = f"Awaiting {len(pending_approvals)} approval(s)"
        elif tasks_completed < tasks_total:
            mission_status = "in_progress"
            completion_reason = f"{tasks_completed}/{tasks_total} tasks completed"
        else:
            mission_status = "completed"
            completion_reason = "Mission execution finished"

        return {
            "mission_report": report_content,
            "report_file_path": report_file_path,
            "mission_status": mission_status,
            "completion_reason": completion_reason,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"report_generation_node: {str(e)}"]
        }
